In [2]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [9]:
## Defining input and target variables 
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Defining data-frames to store results
md1_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})
md2_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})
md3_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})


for i in range(0, 10):
    
    ## Splitting the data 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    #############
    ## Model 1 ##
    #############
    
    md1 = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
    pred1 = md1.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 < 0.1, 0, 1)
    md1_results.loc[i, 'accuracy'] = accuracy_score(Y_test, pred1)
    md1_results.loc[i, 'recall'] = recall_score(Y_test, pred1)
    

In [10]:
md1_results

,accuracy,recall
0,0.431694,0.830357
1,0.409836,0.875000
2,0.442623,0.910714
3,0.446721,0.830357
4,0.442623,0.875000
...,...,...
95,NaN,NaN
96,NaN,NaN
97,NaN,NaN
98,NaN,NaN


In [6]:
md1_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})
md1_results.shape

(100, 2)